## 1. Machine Learning & Neural Networks

__a) Adam Optimizer
Recall the standard Stochastic Gradient Descent update rule:__

$$\theta \leftarrow \theta - \alpha \Delta_\theta J_{\text {minibatch}}(\theta)$$

__where $\theta$ is a vector containing all of the model parameters, $J$ is the loss function, $J_{\text {minibatch}}(\theta)$ is the gradient of the loss function with respect to the parameters on a minibatch of data, and $\alpha$ is the learning rate. Adam Optimization uses a more sophisticated update rule with two additional
steps:__

__i. First, Adam uses a trick called momentum by keeping track of m, a rolling average of the gradients:__

$$m \leftarrow \beta_1 m + (1 - \beta_1) \Delta_\theta J_{\text {minibatch}}(\theta) \\
\theta \leftarrow \theta - \alpha m$$

__where $\beta_1$ is a hyperparameter between 0 and 1 (often set to 0.9). Briefly explain (you don’t need
to prove mathematically, just give an intuition) how using m stops the updates from varying as much and why this low variance may be helpful to learning, overall.__

Answer:
On each iteration in vanilla SGD we are making a step in gradient's direction. Due usage of minibatches these steps can vary a lot and path to optimum can look like a zigzag. When applying more complex update rule incorporating rolling averages of the gradients, we smooth our path. Every step is a sum of two vectors - big fraction of the previous and a small fraction of the new one. Overall it makes SGD to perform less wrong steps and it converges faster.

__ii. Adam also uses adaptive learning rates by keeping track of v, a rolling average of the magnitudes of the gradients:__

$$m \leftarrow \beta_1 m + (1 - \beta_1) \Delta_\theta J_{\text {minibatch}}(\theta) \\
v \leftarrow \beta_2 v + (1 - \beta_2) (\Delta_\theta J_{\text {minibatch}}(\theta) \odot \Delta_\theta J_{\text {minibatch}}(\theta)) \\
\theta \leftarrow \theta - \alpha \odot m / \sqrt {v}$$

__where $\odot$ and $/$ denote elementwise multiplication and division (so $z \odot z$ is elementwise squaring) and $\beta_2$ is a hyperparameter between 0 and 1 (often set to 0.99). Since Adam divides the update by $\sqrt {v}$, which of the model parameters will get larger updates? Why might this help with learning?__

Answer:
Weights which showed stability on a number of iterations (previous $v$ and $m$ are close to new $v$ and $m$) will receive larger updates than those which fluctuate too much between iterations. This works like a filter - we prefer to believe in a stable gradients rather than random caused by minibatch nature.

__b) Dropout is a regularization technique. During training, dropout randomly sets units in the hidden layer $h$ to zero with probability $p_{\text drop}$ (dropping different units each minibatch), and then multiplies $h$ by a constant $\gamma$. We can write this as__

$$h_{\text drop} = \gamma d \circ h$$

__where $d \in \{0,1\}^{D_h}$ ($D_h$ is the size of $h$) is a mask vector where each entry is 0 with probability $p_{\text drop}$ and 1 with probability $(1 − p_{\text drop})$. $\gamma$ is chosen such that the expected value of $h_{\text drop}$ is $h$:__

$$E_{p_{\text drop}}[h_{\text drop}]_i = h_i$$

__for all $i \in \{1, \ldots, D_h\}$.__

__i. What must $\gamma$ equal in terms of $p_{\text drop}$? Briefly justify your answer.__

Answer:
$\gamma$ should be equal to $p_{\text drop}$. This makes flowing values and gradients scale back to the original size like if there was no dropout.

__ii. Why should we apply dropout during training but not during evaluation?__

Answer: 
we don't want to remember all vectors $d$ to repeat our process of dropout in test time.

## 2. Neural Transition-Based Dependency Parsing

__a) Go through the sequence of transitions needed for parsing the sentence “I parsed this sentence correctly”. The dependency tree for the sentence is shown below. At each step, give the configuration of the stack and buffer, as well as what transition was applied this step and what new dependency was added (if any). The first three steps are provided below as an example.__

Answer:



| Stack | Buffer | New Dependency | Transition |
| - | - | - | - |
| ROOT  | I parsed this sentence correctly | - | Init |
| ROOT I | parsed this sentence correctly | -| Shift |
| ROOT I parsed | this sentence correctly | - | Shift |
| ROOT parsed | this sentence correctly | parsed $\rightarrow$ I | Left-Arc |
| ROOT parsed this | sentence correctly | - | Shift |
| ROOT parsed this sentence | correctly | - | Shift |
| ROOT parsed sentence | correctly | sentence $\rightarrow$ this | Left-Arc |
| ROOT parsed | correctly | parsed $\rightarrow$ sentence | Right-Arc |
| ROOT parsed correctly | $\varnothing$ | - | Shift |
| ROOT parsed | $\varnothing$ | parsed $\rightarrow$ correctly | Right-Arc |
| ROOT | $\varnothing$ | ROOT $\rightarrow$ parsed | Right-Arc |

__b) A sentence containing $n$ words will be parsed in how many steps (in terms of $n$)? Briefly
explain why.__

Answer:
Each word can be moved in two ways: from "buffer" to "stack" during "Shift" transition and from "Stack" to "Dependencies" during "Left-Arc" or "Right-Arc" transition. Every movement can be performed only once for every word - algorithm doesn't allow to return word back to any list. So we will perform 2 movements for every word, i.e. $O(2*n)$

__f) In this question are four sentences with dependency parses obtained from a parser. Each sentence
has one error, and there is one example of each of the four types above. For each sentence, state
the type of error, the incorrect dependency, and the correct dependency.__

Answer:

i.

Error type: Verb Phrase Attachment Error

Incorrect dependency: wedding $\rightarrow$ fearing

Correct dependency: heading $\rightarrow$ fearing

ii.

Error type: Coordination Attachment Error

Incorrect dependency: makes $\rightarrow$ rescue

Correct dependency: rush $\rightarrow$ rescue

iii.

Error type: Prepositional Phrase Attachment Error

Incorrect dependency: named $\rightarrow$ Midland

Correct dependency: guy $\rightarrow$ Midland

iv.

Error type: Modifier Attachment Error

Incorrect dependency: elements $\rightarrow$ most

Correct dependency: crucial $\rightarrow$ most

Results of model evaluation on a test set:
```
Final evaluation on test set
2919736it [00:00, 57730250.86it/s]                        
- test UAS: 89.04
```